In [13]:
!pip install transformers accelerate wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import wandb

In [2]:
df = pd.read_csv('data_logs/wmt14_bleu_threshold.csv')
df['labels'] = df[['1b', '3b', '8b']].values.tolist()
df['labels'] = df['labels'].apply(lambda x: [int(i) for i in x])

In [3]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['input_text'].tolist(),
    df['labels'].tolist(),
    test_size=0.2,
    random_state=42
)

In [4]:
df.head()

,input_text,1b,3b,8b,labels
0,Wiederaufnahme der Sitzungsperiode,0,0,1,"[0, 0, 1]"
1,"Ich erkläre die am Freitag, dem 17. Dezember u...",1,1,1,"[1, 1, 1]"
2,"Wie Sie feststellen konnten, ist der gefürchte...",0,0,1,"[0, 0, 1]"
3,Im Parlament besteht der Wunsch nach einer Aus...,0,1,1,"[0, 1, 1]"
4,Heute möchte ich Sie bitten - das ist auch der...,1,1,1,"[1, 1, 1]"


In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
class WMT14Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    def __len__(self):
        return len(self.labels)

In [7]:
train_dataset = WMT14Dataset(train_encodings, train_labels)
val_dataset = WMT14Dataset(val_encodings, val_labels)

In [8]:
len(train_dataset), len(val_dataset)

(4000, 1000)

In [9]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=3, problem_type="multi_label_classification"
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir='./data_logs/distillbert',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_steps=1,
    logging_dir='./data_logs/distillbert',
    report_to="wandb" 
)

/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
wandb.init(project="MESS+", name="distillbert-multi-label-run")
trainer.train()
wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: ryanzhangofficial (ryzhangofficial). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to av

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.499200,0.419559


In [29]:
import os
import pickle

save_directory = "data_logs/distillbert"
os.makedirs(save_directory, exist_ok=True)

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

with open(os.path.join(save_directory, "multi-label-binarizer.pkl"), "wb") as f:
    pickle.dump(model, f)

print(f"Model, tokenizer, and multi-label binarizer saved in {save_directory}.")

Model, tokenizer, and multi-label binarizer saved in data_logs/distillbert.
